In [31]:
import pandas as pd
import geopandas
from shapely.geometry import Point
import matplotlib.pyplot as plt

import folium
from folium.plugins import MarkerCluster

In [32]:
# reads & stores the train data in a pandas dataframe
df_train = pd.read_csv("/Users/nialdaly/Documents/ey_nextwave/data/data_train.csv", index_col=False)

# drops first (unneccessary column)
df_train = df_train.drop(df_train.columns[0], axis=1)

# 
df_train = df_train.sample(n=500, random_state=42)

In [42]:
from pyproj import Proj, transform

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
# x1,y1 = -11705274.6374,4826473.6922
x1, y1 = -1.913738e+07, 3.757411e+06
x2,y2 = transform(inProj,outProj,x1,y1)
print(x2,y2)

-171.91400952003247 31.95504476886395


In [37]:
df_train['coords_entry'] = list(zip(df_train.x_entry, df_train.y_entry))

# 
# df_train['coords_entry'] = df_train['coords_entry'].apply(Point)

df_train.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,coords_entry
440102,8aa0b65bc05526668718871bd8cd1f2a_5,traj_8aa0b65bc05526668718871bd8cd1f2a_5_1,01:05:18,01:05:18,0.0,0.0,0.0,3.757411e+06,-1.913738e+07,3.757411e+06,-1.913738e+07,"(3757411.30054168, -19137382.59558711)"
438577,8a21f14cbf2a4ec002c716a259e9b6bc_5,traj_8a21f14cbf2a4ec002c716a259e9b6bc_5_2,10:40:56,10:40:56,NaN,NaN,NaN,3.756545e+06,-1.918472e+07,3.756545e+06,-1.918472e+07,"(3756545.234903308, -19184716.578236856)"
712629,e0571d1a8bd3602ae313d02e3877b321_31,traj_e0571d1a8bd3602ae313d02e3877b321_31_13,11:00:15,11:02:38,NaN,NaN,NaN,3.760233e+06,-1.934043e+07,3.757173e+06,-1.923925e+07,"(3760233.24963329, -19340427.40128488)"
678461,d53c40703f6f88dbe5c7aa1c85e3f874_29,traj_d53c40703f6f88dbe5c7aa1c85e3f874_29_1,15:34:42,15:34:42,NaN,NaN,NaN,3.759116e+06,-1.925871e+07,3.759116e+06,-1.925871e+07,"(3759115.601945725, -19258710.15883579)"
160062,31ff6899956ced08fa358f32a3193647_29,traj_31ff6899956ced08fa358f32a3193647_29_5,15:15:47,15:15:47,-1.0,-1.0,-1.0,3.757721e+06,-1.924037e+07,3.757721e+06,-1.924037e+07,"(3757720.768726085, -19240373.640319847)"


In [38]:


# creates the GeoDataFrame by setting geometry with the coordinates created previously
gdf = geopandas.GeoDataFrame(df_train, crs={'init' :'epsg:4326'}, geometry='coords_entry')

gdf.head()

TypeError: Input geometry column must contain valid geometry objects.

In [39]:


# all records between 3 and 4pm
# df_train = df_train[(df_train['time_entry'] >= '15:00:00') & (df_train['time_exit'] <= '16:00:00')]

m = folium.Map(location=[51.5, -0.1], zoom_start=10)

icons = [folium.Icon(icon="car", prefix="fa") for _ in range(len(df_train['coords_entry']))]

cluster = MarkerCluster(locations=df_train['coords_entry'], icons=icons)
m.add_child(cluster)
m